<a href="https://colab.research.google.com/github/1Pampu/BigData-ML-Proyectos/blob/main/TP1/Big_Data_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import de librerias, carga de datos y filtrado de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os

In [ ]:
ruta_de_archivos =  "/content/drive/MyDrive/Dataset de ventas2"
archivos = os.listdir(ruta_de_archivos)

dataframes= {}

for archivo in archivos:
    if archivo.endswith('.csv'):                                                 # Asegurarse de que solo se consideren archivos CSV
        mes = archivo.split('_')[3].split('.')[0]                                # Obtener el nombre del mes
        ruta_completa = os.path.join(ruta_de_archivos, archivo)                  # Ruta completa al archivo
        df = pd.read_csv(ruta_completa)                                          # Cargar el archivo CSV en un DataFrame
        dataframes[mes] = df                                                     # Agregar el DataFrame al diccionario usando el nombre del mes

In [ ]:
for key in dataframes.keys():
  # Identificar filas con errores y transformarlas a NaN
  dataframes[key]["ID de Pedido"] = pd.to_numeric(dataframes[key]["ID de Pedido"], errors = "coerce")
  dataframes[key]["Cantidad Pedida"] = pd.to_numeric(dataframes[key]["Cantidad Pedida"], errors = "coerce")
  dataframes[key]["Precio Unitario"] = pd.to_numeric(dataframes[key]["Precio Unitario"], errors = "coerce")

  # Filtrar filas NaN
  dataframes[key] = dataframes[key].loc[~dataframes[key]["ID de Pedido"].isna()]
  dataframes[key] = dataframes[key].loc[~dataframes[key]["Cantidad Pedida"].isna()]
  dataframes[key] = dataframes[key].loc[~dataframes[key]["Precio Unitario"].isna()]

  # Eliminar valores NaN
  dataframes[key] = dataframes[key].dropna()

  # Convertir valores
  dataframes[key]["ID de Pedido"] = dataframes[key]["ID de Pedido"].astype(int)
  dataframes[key]["Cantidad Pedida"] = dataframes[key]["Cantidad Pedida"].astype(int)
  dataframes[key]["Precio Unitario"] = dataframes[key]["Precio Unitario"].astype(int)
  dataframes[key]["Fecha de Pedido"] = pd.to_datetime(dataframes[key]["Fecha de Pedido"])

  # Extraemos datos mas especificos
  dataframes[key]["Hora"] = dataframes[key]["Fecha de Pedido"].dt.hour
  dataframes[key]["Dia"] = dataframes[key]["Fecha de Pedido"].dt.day

  # Separo la direccion de envio en calle ciudad y PO BOX
  direccion_df = dataframes[key]['Dirección de Envio'].str.split(', ',expand=True)
  direccion_df.columns = ['Calle','Ciudad','PO BOX']

  # Separo el PO BOX en Estado y Cod Postal
  direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ',expand=True)
  dataframes[key] = pd.concat([dataframes[key],direccion_df],axis=True)


In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
      'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Ordenamos las keys del diccionario en el orden de los meses
dictOrdenado = {}
for mes in meses:
  dictOrdenado[mes] = dataframes[mes]
dataframes = dictOrdenado

# Comportamiento de las ventas en los distintos meses

In [ ]:
# Extraemos informacion a graficar
ventasPorMes = []
ingresosPorMes = []

for key in dataframes.keys():
  ventaMes = dataframes[key]["Cantidad Pedida"].sum()
  ventasPorMes.append(ventaMes)
  ingresoMes = (dataframes[key]["Cantidad Pedida"] * dataframes[key]["Precio Unitario"]).sum()
  ingresosPorMes.append(ingresoMes)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = meses,
    y = ventasPorMes,
    marker = dict(color = ventasPorMes, colorscale = "reds", colorbar = dict(title = "Total de Ventas", x = 1.0, thickness = 15)),
    text = ventasPorMes,
    textposition = 'auto',
))

fig.update_layout(
    title = "Ventas por mes",
    xaxis = dict(title = "Mes"),
    yaxis = dict(title = "Total de Ventas"),
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = meses,
    y = ingresosPorMes,
    marker = dict(color = ingresosPorMes, colorscale = "hot", colorbar = dict(title = "Total de Ingresos", x=1.0, thickness=15)),
    text = ingresosPorMes,
    textposition = 'auto',
))

fig.update_layout(
    title = "Ingresos por mes",
    xaxis = dict(title = "Mes"),
    yaxis = dict(title = "Total de Ingresos"),
)

fig.show()

# Optimización de la publicidad y patrón de ventas por hora

In [ ]:
# Extraemos datos a graficar
ventasXdiaXmes = []

for mes in meses:
  ventasXhora = dataframes[mes].groupby('Hora')['Cantidad Pedida'].sum()
  ventasXdiaXmes.append(ventasXhora)

df = pd.DataFrame(ventasXdiaXmes)         # Convertimos la lista de series en dataframe

ventasXhora= []
for i in range(0,24):
  df[i].sum()
  ventasXhora.append(df[i].sum())


Horas = list(range(24))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x = Horas,
        y = ventasXhora,
        mode = 'lines',
        name = mes,
        line = dict(width = 2,shape = 'spline'),
    ))

fig.update_layout(
    title = "Total de unidades vendidas por hora",
    xaxis = dict(title = "Horas", tickvals = Horas),
    yaxis = dict(title = "Unidades vendidas", side = "left"),
    legend = dict(y = 0.3, yanchor = 'bottom'),
)

fig.show()

In [ ]:
fig = go.Figure()
i = 0
for mes in meses:
    # Agrego un trazado por mes
    fig.add_trace(go.Scatter(
        x = Horas,
        y = ventasXdiaXmes[i],
        mode = 'lines',
        name = mes,
        line = dict(width = 2, shape = 'spline'),
    ))

    i += 1


fig.update_layout(
    title = "Unidades vendidas por hora a lo largo de los meses",
    xaxis = dict(title = "Horas", tickvals = Horas),
    yaxis = dict(title = "Unidades vendidas", side = "left"),
    legend = dict(y = 0.3, yanchor = 'bottom'),
)

fig.show()

In [ ]:
horasPico = [12,15,19]

fig = go.Figure()

for hora in horasPico:
    totalProductosVendidos = []

    # Calcular la cantidad total de productos vendidos para la hora actual
    for mes in meses:
        dataframeMes = dataframes[mes]
        ventasHoraPico = dataframeMes[dataframeMes['Hora'] == hora]
        totalProductosVendidos.append(ventasHoraPico['Cantidad Pedida'].sum())

    # Agrego un trazado por horario pico
    fig.add_trace(go.Scatter(
        x = meses,
        y = totalProductosVendidos,
        mode = 'lines',
        name = f'{hora}Hs',
        line = dict(width = 2, shape = 'spline'),
    ))

fig.update_layout(
    title = "Unidades vendidas en las horas pico",
    xaxis = dict(title = "Meses"),
    yaxis = dict(title = "Unidades vendidas", side = "left"),
    legend = dict(y = 0.3, yanchor = 'bottom'),
)

fig.show()

# Distribución de ventas por ubicación

In [ ]:
# Calculamos los pedidos por estado y pedidos por estado en los meses
pedidosXestadoXmes = []
for mes in meses:
  pedidosXestadoXmes.append(dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum())

pedidosXestado = pd.concat(pedidosXestadoXmes)
pedidosXestado = pedidosXestado.groupby(level = 0).sum()
pedidosXestado = pedidosXestado.to_frame()
pedidosXestado = pedidosXestado.reset_index(inplace = False)

# Hacemo un dataframe por estado y a cada uno le agregamos una lista con sus ventas x mes
statesDataframe = {}
for mes in pedidosXestadoXmes:
  mes = mes.reset_index(inplace = False)
  for index,fila in mes.iterrows():
    try:
       statesDataframe[fila["Estado"]].append(fila["Cantidad Pedida"])
    except:
      statesDataframe[fila["Estado"]] = []
      statesDataframe[fila["Estado"]].append(fila["Cantidad Pedida"])


In [ ]:
fig = px.choropleth(pedidosXestado, locations = 'Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="speed",
                           scope="usa",
                          title= 'Ventas por estado'
                  )

fig.show()

In [ ]:
fig = go.Figure()

i = 0
for array in pedidosXestadoXmes:
  array = array.reset_index(inplace = False)

  # Agrego un trazado por mes
  fig.add_trace(go.Scatter(
        x = array['Estado'],
        y = array['Cantidad Pedida'],
        mode = 'lines',
        name = meses[i],
        line = dict(width = 2, shape = 'spline'),
))
  i += 1

fig.update_layout(
    title = "Variacion de las ventas a lo largo de los meses por Estado",
    xaxis = dict(title = "Estados"),
    yaxis = dict(title = "Unidades vendidas", side = "left"),
    legend = dict(y = 0.3, yanchor = 'bottom'),
)

fig.show()

In [ ]:
fig = go.Figure()

for estado,ventas in statesDataframe.items():

  # Agrego un trazado por Estado
  fig.add_trace(go.Scatter(
        x = meses,
        y = ventas,
        mode = 'lines',
        name = estado,
        line = dict(width = 2, shape = 'spline'),
))

fig.update_layout(
    title = "Variacion de las ventas a lo largo de los meses por Estado",
    xaxis = dict(title = "Estados"),
    yaxis = dict(title = "Unidades vendidas", side = "left"),
    legend = dict(y = 0.3, yanchor = 'bottom'),
)

fig.show()

# Análisis del producto más vendido

In [ ]:
# Extraemos y agrupamos los productos mas vendidos
productosMasVendidosXmes = {}
for mes in meses:
  productos = dataframes[mes].groupby('Producto').agg({'Cantidad Pedida' : 'sum', 'Precio Unitario':'first'})
  productosMasVendidosXmes[mes] = productos

# Lista de el producto mas exitoso del mes
productoDelMes = []
cantidadDelProducto = []
for value in productosMasVendidosXmes.values():
  value = value.reset_index(inplace = False)
  cantidadDelProducto.append(value["Cantidad Pedida"].max())
  index = value["Cantidad Pedida"].idxmax()
  producto = value.loc[index, 'Producto']
  productoDelMes.append(producto)

# Mas vendidos en total
productosMasVendidosTotal = pd.DataFrame()
for value in productosMasVendidosXmes.values():
  value = value.reset_index(inplace = False)
  productosMasVendidosTotal = pd.concat([productosMasVendidosTotal,value])
productosMasVendidosTotal = productosMasVendidosTotal.groupby('Producto').agg({'Cantidad Pedida' : 'sum', 'Precio Unitario':'first'}).reset_index(inplace = False)

# Ordenamos de mayor a menor
productosMasVendidosTotal = productosMasVendidosTotal.sort_values(by = 'Cantidad Pedida', ascending = False)


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = productosMasVendidosTotal['Producto'],
    y = productosMasVendidosTotal['Cantidad Pedida'],
    marker = dict(color = productosMasVendidosTotal['Cantidad Pedida'], colorscale = "speed"),
    text = productosMasVendidosTotal['Cantidad Pedida'],
    textposition = 'auto',
    yaxis = "y",
    name = 'Total de Ventas'
))

fig.add_trace(go.Scatter(
        x = productosMasVendidosTotal['Producto'],
        y = productosMasVendidosTotal['Precio Unitario'],
        name = 'Precio unitario',
        line = dict(color = 'Red',shape = 'spline', smoothing = 1.3, width = 3),
        yaxis = 'y2'
        ))

fig.update_layout(
    title = "Productos mas vendidos en total",
    xaxis = dict(title = "Productos"),
    yaxis = dict(title = "Cantidad de Pedidos"),
    yaxis2 = dict(title = "Precio Unitario", overlaying = "y", side = "right"),
    yaxis_range = [0, max(productosMasVendidosTotal['Cantidad Pedida']) * 1.1],
    yaxis2_range = [0, max(productosMasVendidosTotal['Precio Unitario']) * 1.1]
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = meses,
    y = cantidadDelProducto,
    marker = dict(color = cantidadDelProducto, colorscale = "turbid", colorbar = dict(title="Total de Ventas", x=1.0, thickness=15)),
    text = productoDelMes,
    textposition = 'auto',
))

fig.update_layout(
    title = "Productos mas exitosos por cada mes",
    xaxis = dict(title = "Meses"),
    yaxis = dict(title = "Cantidad Pedida"),
)

fig.show()

# Tendencia ventas

In [ ]:
# Ventas En los distintos dias
listas = {}
mesAnterior = 0

for key,fila in dataframes.items():
  ventasMes = dataframes[key].groupby('Dia')['Cantidad Pedida'].sum().astype('int')
  ventasMes = ventasMes.reset_index(inplace = False)
  try:
    mesAnterior += ventasMes
  except:
    mesAnterior = ventasMes

ventasPorDia = mesAnterior.dropna()

In [ ]:
fig = go.Figure()

for key, item in dataframes.items():

  Compras_dia = dataframes[key].groupby('Dia')['Cantidad Pedida'].count().astype('int')
  Dias = list(Compras_dia.index)

  fig.add_trace(go.Scatter(
      x = Dias,
      y = Compras_dia,
      mode = 'lines',
      name = key,
      line = dict( shape = 'spline', smoothing = 1.3, width = 3)))

fig.update_layout(
    title = "Unidades vendidas en los diferentes meses",
    xaxis = dict(title = "Dia", tickvals = Dias),
    yaxis = dict(title = "Unidades vendidas", side = "left"))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = Dias,
    y = ventasPorDia["Cantidad Pedida"],
    mode = 'lines',
    name = "Ventas por hora",
    line = dict(color = "red", shape = 'spline', smoothing = 1.3, width = 3)))

fig.update_layout(
    title = "Unidades vendidas en todos los meses",
    xaxis = dict(title = "Dia", tickvals = Dias),
    yaxis = dict(title = "Unidades vendidas", side = "left"))

fig.show()